In [35]:
import numpy as np
import pandas as pd
import os
import requests
from tqdm import tqdm
import time as tm

In [ ]:
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 

In [37]:
df = pd.read_excel('Cluster/Faenas25.xlsx')

In [38]:
df.columns = ['RutEmpresa' if col == 'RutEmpresa' else col.lstrip() for col in df.columns]

# Limpiar espacios dentro de las columnas (excepto RutEmpresa)
for col in df.select_dtypes(include='object').columns:
    if col != 'RutEmpresa':
        df[col] = df[col].str.strip()

In [39]:
exclude_states = [
    "ABANDONADA",
    "EXPLORACIÓN FINALIZADA",
    "EN REVISIÓN DE PROYECTO",
    "CIERRE FINAL",
    "PROYECTADA"
]

df = df[~df["Estado"].str.upper().str.strip().isin(exclude_states)]


In [40]:
# === Your city coordinates ===
cities_coords = {
    "Santiago": (-33.4489, -70.6693),
    "Valparaíso": (-33.0472, -71.6127),
    "Viña_del_Mar": (-33.0153, -71.5500),
    "Antofagasta": (-23.6500, -70.4000),
    "La_Serena": (-29.9037, -71.2489),
    "Coquimbo": (-29.9533, -71.3395),
    "Rancagua": (-34.1708, -70.7400),
    "Talca": (-35.4261, -71.6550),
    "Iquique": (-20.2307, -70.1350),
    "Arica": (-18.4783, -70.3126),
    "Calama": (-22.4624, -68.9270),
    "Copiapó": (-27.3668, -70.3326),
    "San_Bernardo": (-33.6000, -70.7167),
    "Curicó": (-34.9853, -71.2410),
    "Ovalle": (-30.6000, -71.2000)
}

# Example: your dataframe
# df = pd.read_csv('your_dataset.csv')

# === Prepare new columns ===
for city in cities_coords.keys():
    df[f"dist_{city}_km"] = None
    df[f"time_{city}_min"] = None

# === OSRM function ===
def get_osrm_distance_time(lat1, lon1, lat2, lon2):
    """Query OSRM for distance (km) and duration (min)."""
    url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()
            route = data['routes'][0]
            dist_km = route['distance'] / 1000  # meters → km
            dur_min = route['duration'] / 60    # seconds → minutes
            return dist_km, dur_min
        else:
            return None, None
    except Exception:
        return None, None

# === Iterate through dataset with progress bar ===
print("Calculating OSRM distances and travel times...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    lat, lon = row["Latitud"], row["Longitud"]

    for city, (city_lat, city_lon) in cities_coords.items():
        d, t = get_osrm_distance_time(lat, lon, city_lat, city_lon)
        df.at[idx, f"dist_{city}_km"] = d
        df.at[idx, f"time_{city}_min"] = t
    
    tm.sleep(0.1)  
    
print("✅ Finished adding 15 distance and time columns.")

# --- Save result ---
df.to_csv("df_with_osrm_distances_times.csv", index=False)

Calculating OSRM distances and travel times...


  0%|          | 0/15160 [00:00<?, ?it/s]

100%|██████████| 15160/15160 [67:09:48<00:00, 15.95s/it]       


✅ Finished adding 15 distance and time columns.
